<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/DANSE_Mar26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Step-by-Step Workflow to Apply DANES Methodology on `facebook-fact-check.csv` Dataset**

---

## **📌 Step 1: Install Required Libraries**

---

---
## **📌 Step 2: Load and Explore the Dataset**
Start by loading and understanding your dataset.

In [3]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/facebook-fact-check.csv", encoding='latin1')

# Check initial shape and missing values
print("Initial shape:", df.shape)  # Should be (2282, 14)
print("\nMissing values:\n", df.isnull().sum())

Initial shape: (2282, 13)

Missing values:
 account_id           0
post_id              0
Category             0
Page                 0
Post URL             0
Date Published       0
Post Type            0
Rating               0
Debate            1984
share_count         70
reaction_count       2
comment_count        2
Context Post         0
dtype: int64


### 🔹 **Check for Missing Values**
---

---
## **📌 Step 3: Define Target Variable**
- If `Rating` column contains fact-checking labels, convert it into a binary/multi-class target variable.
---

In [5]:
# Check unique ratings
print("Unique Ratings:", df['Rating'].unique())

# Example mapping (adjust based on your data)


# For multi-class, use LabelEncoder instead:
from sklearn.preprocessing import LabelEncoder
df['label'] = LabelEncoder().fit_transform(df['Rating'])

Unique Ratings: ['no factual content' 'mostly true' 'mixture of true and false'
 'mostly false']


---

## **📌 Step 4: Preprocess Text Data (Text Branch)**
Since DANES uses deep learning models, we need to **clean, tokenize, and embed** the text.

In [7]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

df['processed_text'] = df['Context Post'].apply(preprocess_text)

# Train Word2Vec model
w2v_model = Word2Vec(df['processed_text'], vector_size=100, window=5, min_count=1, workers=4)

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['processed_text'].apply(lambda x: ' '.join(x)))
sequences = tokenizer.texts_to_sequences(df['processed_text'].apply(lambda x: ' '.join(x)))
max_len = 100  # Adjust based on data analysis
X_text = pad_sequences(sequences, maxlen=max_len, padding='post')

# Create embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### 🔹 **Convert Text to Word Embeddings**
Use **Word2Vec, FastText, or GloVe** to obtain word embeddings.

---


---

## **📌 Step 5: Prepare Social Context Features (Social Branch)**
We normalize numerical social features (`share_count`, `reaction_count`, `comment_count`).

---

---
## **📌 Step 6: Train-Test Split**

---

---
## **📌 Step 7: Build the DANES Model (Deep Learning)**
 Create the **Text Branch (LSTM)** and **Social Branch (MLP/CNN)** and combine them.

---

---
## **📌 Step 8: Train the Model**

---

## **📌 Step 9: Evaluate the Model**

---

## **📌 Step 10: Save the Model for Future Use**

---

Let's proceed with implementing Steps 3 to 10:

---

## **📌 Step 3: Define Target Variable**
Convert the `Rating` column into a binary target variable.

```python
# Check unique ratings
print("Unique Ratings:", df['Rating'].unique())

# Example mapping (adjust based on your data)
# Binary classification: 'True' vs others
df['label'] = df['Rating'].apply(lambda x: 1 if x == 'True' else 0)

# For multi-class, use LabelEncoder instead:
# from sklearn.preprocessing import LabelEncoder
# df['label'] = LabelEncoder().fit_transform(df['Rating'])
```

---

## **📌 Step 4: Preprocess Text Data**
Clean the text and convert it into embeddings.

```python
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

df['processed_text'] = df['Context Post'].apply(preprocess_text)

# Train Word2Vec model
w2v_model = Word2Vec(df['processed_text'], vector_size=100, window=5, min_count=1, workers=4)

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['processed_text'].apply(lambda x: ' '.join(x)))
sequences = tokenizer.texts_to_sequences(df['processed_text'].apply(lambda x: ' '.join(x)))
max_len = 100  # Adjust based on data analysis
X_text = pad_sequences(sequences, maxlen=max_len, padding='post')

# Create embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
```

---

## **📌 Step 5: Prepare Social Context Features**
Handle missing values and scale numerical features.

```python
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Impute missing values
imputer = SimpleImputer(strategy='median')
social_features = imputer.fit_transform(df[['share_count', 'reaction_count', 'comment_count']])

# Scale features
scaler = StandardScaler()
X_social = scaler.fit_transform(social_features)
```

---

## **📌 Step 6: Train-Test Split**
Split the dataset into training and testing sets.

```python
from sklearn.model_selection import train_test_split

X_text_train, X_text_test, X_social_train, X_social_test, y_train, y_test = train_test_split(
    X_text, X_social, df['label'], test_size=0.2, stratify=df['label'], random_state=42)
```

---

## **📌 Step 7: Build the DANES Model**
Create a dual-input neural network.

```python
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate

# Text branch
text_input = Input(shape=(max_len,))
x = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(text_input)
x = LSTM(64)(x)

# Social branch
social_input = Input(shape=(3,))
y = Dense(32, activation='relu')(social_input)

# Combine branches
combined = Concatenate()([x, y])
z = Dense(64, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[text_input, social_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
```

---

## **📌 Step 8: Train the Model**
Train the model on the training data.

```python
history = model.fit(
    [X_text_train, X_social_train], y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
```

---

## **📌 Step 9: Evaluate the Model**
Assess performance on the test set.

```python
# Evaluate accuracy
loss, accuracy = model.evaluate([X_text_test, X_social_test], y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Generate classification report
from sklearn.metrics import classification_report
y_pred = (model.predict([X_text_test, X_social_test]) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
```

---

## **📌 Step 10: Save the Model**
Save the trained model for future use.

```python
model.save('danes_model.h5')
```

---

This completes the implementation of the DANES framework on your dataset. Adjust hyperparameters (epochs, embedding dimensions, etc.) based on your specific requirements.